In [1]:
import random

In [2]:
def mask_sentence(tokens, tokenizer):
    """
    Masking some random tokens for Language Model task with probabilities as in the original BERT paper.
    :param tokens: list of str, tokenized sentence.
    :param tokenizer: Tokenizer, object used for tokenization (we need it's vocab here)
    :return: (list of str, list of int), masked tokens and related labels for LM prediction
    """
    output_label = []

    for i, token in enumerate(tokens):
        prob = random.random()
        # mask token with 15% probability
        if prob < 0.15:
            prob /= 0.15

            # 80% randomly change token to mask token
            if prob < 0.8:
                tokens[i] = "[MASK]"

            # 10% randomly change token to random token
            elif prob < 0.9:
                tokens[i] = random.choice(list(tokenizer.vocab.items()))[0]

            # -> rest 10% randomly keep current token

            # append current token to output (we will predict these later)
            try:
                output_label.append(tokenizer.vocab[token])
            except KeyError:
                # For unknown words (should not occur with BPE vocab)
                output_label.append(tokenizer.vocab["[UNK]"])
                print("Cannot find token '{}' in vocab. Using [UNK] insetad".format(token))
        else:
            # no masking token (will be ignored by loss function later)
            output_label.append(-1)

    return tokens, output_label



In [3]:
print(tokenizer.encode(f"Hello, I am a great guy", add_special_tokens=True))
len(tokenizer.encode(f"Hello, I am a great guy", add_special_tokens=True))

NameError: name 'tokenizer' is not defined

In [4]:
model.parameters

NameError: name 'model' is not defined

In [23]:
text = "America saw the third highest daily increase of new cases on Sunday. The president is ashamed to mention the casualities."

def tokenize_text(text):
    # Tokenize input
    tokenized_text = tokenizer.tokenize(text)
    return tokenized_text

In [24]:
random_word(list(tokenize_text(text)),tokenizer)

(['[MASK]',
  'saw',
  'the',
  'heiress',
  'highest',
  'daily',
  'increase',
  '[MASK]',
  'new',
  'cases',
  'on',
  'sunday',
  '.',
  'the',
  'president',
  'is',
  'ashamed',
  'to',
  '[MASK]',
  'the',
  'casual',
  '##ities',
  '.'],
 [2637,
  -1,
  -1,
  2353,
  -1,
  -1,
  -1,
  1997,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  5254,
  -1,
  -1,
  -1,
  -1])

In [36]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
# logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "America saw the third highest daily increase of new diseases on Sunday. The president is ashamed to mention the casualities."
tokenized_text = tokenizer.tokenize(text)

tokenized_text, output_labels = mask_sentence(tokenized_text,tokenizer)
tokenized_text = ['[CLS]'] + tokenized_text + ['[SEP]']
print(tokenized_text)

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
# segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
segments_ids = [0]* len(tokenized_text)

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

AttributeError: 'list' object has no attribute 'replace'

In [35]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)
# assert predicted_token == 'henson'

"


In [28]:
outputs[0].shape

torch.Size([1, 24, 30522])

In [20]:
for i in range(len(tokenized_text)):
    predicted_index = torch.argmax(predictions[0, i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_token)

.
america
saw
the
third
highest
daily
increase
of
new
diseases
on
.
.
the
president
is
ashamed
to
mention
the
fact
.
.
